In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 데이터 저장용 리스트
data = []

# 종목명과 종목코드 딕셔너리
stocks = {
    "오리엔탈바이오": "002630",
    "오리엔탈정공": "014940",
    "CS": "000590",
    "동신건설": "013360",
    "일성건설": "013360",
    "이스타코": "015020",
    "형지I&C": "011080",
    "형지엘리트": "093240",
    "토탈소프트": "045340",
    "서린바이오": "038070",
    "프리엠스": "053160",
    "코나아이": "052400",
    "나우IB": "293580",
}
# 요청 헤더 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

# 페이지 번호 순차적으로 접근
file_name = '나우우1.xlsx'

for page in range(1, 5):  # 1부터 5페이지까지
    url = f"https://finance.naver.com/item/frgn.naver?code=293580&page={page}"

    response = requests.get(url, headers=headers)
    
    # 상태 코드 확인
    if response.status_code == 200:
        # HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 'type2' 테이블에서 모든 행 가져오기
        rows = soup.select("table.type2 tr")
        
        for row in rows:
            cols = row.select("td")  # 열(td) 태그 선택
            
            # 필요한 데이터가 포함된 행인지 확인
            if len(cols) >= 9:  # 최소 9개의 열이 있어야 모든 데이터 추출 가능
                date = cols[0].text.strip()  # 날짜
                close_price = cols[1].text.strip()  # 종가
                change_amount = cols[2].text.strip()  # 전일 대비
                change_rate = cols[3].text.strip().replace('%','')  # 변동률
                volume = cols[4].text.strip()  # 거래량
                foreign_net = cols[5].text.strip()  # 외국인 순매매
                foreign_cum = cols[6].text.strip()  # 외국인 누적
                institution_net = cols[7].text.strip()  # 기관 순매매
                foreign_rate = cols[8].text.strip()  # 외국인 지분율
                
                # 데이터 저장
                data.append([date, close_price, change_amount, change_rate, volume, foreign_net, foreign_cum, institution_net, foreign_rate])
    else:
        print(f"페이지 {page}를 불러오지 못했습니다. 상태 코드:", response.status_code)

# 데이터 역순으로 변경
data.reverse()  # <- 이 부분 추가

# 데이터프레임으로 저장 (선택 사항)
columns = ['날짜', '종가', '전일 대비', '변동률', '거래량', '외국인 순매매', '외국인 누적', '기관 순매매', '외국인 지분율']
df = pd.DataFrame(data, columns=columns)

# 데이터프레임 확인
print(df)

# 엑셀 파일로 저장
df.to_excel(file_name, index=False)
print(f"모든 데이터가 '{file_name}'로 저장되었습니다.")


            날짜     종가           전일 대비    변동률        거래량  외국인 순매매    외국인 누적  \
0   2024.09.27  1,012   하락\n\t\t\t\t3  -0.30     69,749   +1,123   -16,140   
1   2024.09.30  1,010   하락\n\t\t\t\t2  -0.20    134,632      +21   -42,654   
2   2024.10.02    991  하락\n\t\t\t\t19  -1.88     71,262   -1,180   -32,667   
3   2024.10.04    983   하락\n\t\t\t\t8  -0.81     91,228   -2,168    -9,387   
4   2024.10.07    980   하락\n\t\t\t\t3  -0.31    106,295   -2,323   -23,692   
..         ...    ...             ...    ...        ...      ...       ...   
75  2025.01.20  1,189  하락\n\t\t\t\t36  -2.94  1,072,536  -21,986   -66,389   
76  2025.01.21  1,168  하락\n\t\t\t\t21  -1.77    853,218        0  +140,718   
77  2025.01.22  1,144  하락\n\t\t\t\t24  -2.05    699,919        0    -2,283   
78  2025.01.23  1,138   하락\n\t\t\t\t6  -0.52    496,868        0   +16,598   
79  2025.01.24  1,134   하락\n\t\t\t\t4  -0.35  1,882,294        0  -100,934   

     기관 순매매 외국인 지분율  
0   393,862   0.41%  
1   351,208   0.37%

: 